**Basic Analysis**
* Number of user queries
* Number of total characters (unicode?)
    * Number of *human languages* among all conversations
* Number of *Tools*
    * Number of Multimedia
        * from: users/assistants
        * type: image/file/\<code\>
* Size of HTML

**More conversations?** Chatgpt is known to spit too much text.

**Other Implications**

Other things to measure? People switching between models? I don't know what this means in the underlying system...

This is by nature dataset of LLM generated language in a conversation...

In [1]:
import os
import json

In [7]:
conversation_fnames = os.listdir('conversations')
conversation_fnames = [fname for fname in conversation_fnames if '.json' in fname]

In [86]:
conversation_example = conversation_fnames[0]
with open(f'conversations/{conversation_example}', 'r') as f:
    conversation = json.load(f)

conversation

{'url': 'https://chatgpt.com/share/675d357a-f284-800a-82d6-22ebc20f93ed',
 'title': '生物の進化と環境',
 'conversation': [{'role': 'system', 'message': ['']},
  {'role': 'user',
   'message': ['Original custom instructions no longer available']},
  {'role': 'user',
   'message': ['こんばんは、『生物における周辺環境へのアプローチの変化をどこまで進化と判断するか』というテーマで議論しましょう。']},
  {'role': 'assistant', 'message': []},
  {'role': 'assistant',
   'message': ['こんばんは！そのテーマは興味深いですね。まず、「進化」とは何かについて整理しましょう。一般的に、生物の進化は世代を経て遺伝子が変化し、特定の環境に対して適応する過程を指します。しかし、周辺環境へのアプローチの変化には、学習や文化的な変遷、行動の適応などが含まれ、それを進化と呼ぶべきかどうかは議論の余地があります。\n\n例えば、道具の使用や協力行動といった行動面での変化は、遺伝的な変異によるものではなく学習や模倣によるものが多い。しかし、それが集団全体に広がり、次世代にも伝わるとしたら、ある種の「文化進化」とも言えるでしょう。ここでの問題は、遺伝的進化だけを「進化」と呼ぶのか、文化的・行動的変化も進化とみなすべきか、という点です。\n\n進化の定義を遺伝的な変化だけに限定すべきだとすると、環境への行動的なアプローチの変化は進化ではないという立場になりますが、あなたの考えはどうでしょう？この境界線はかなり曖昧ですよね。']},
  {'role': 'user',
   'message': ['確かダーウィンフィンチはガラパゴス諸島の環境により様々な変化をしていて進化論の一モデルとして学んだ覚えがあるのですがその中には小枝を使って木の幹から虫を採る種もいたと記憶しています。これは他の進化を遂げたフィンチから見てどういう立ち位置になるのでしょう？']},

In [87]:
"""
Count the number of backNforth in a conversation
"""

roles = ['user', 'assistant']
backNforth = 0
prev_role = 'user'
for take in conversation['conversation']:
    if take['role'] == 'user':
        prev_role = 'user'
    elif take['role'] == 'assistant' and prev_role == 'user':
        prev_role = 'assistant'
        backNforth += 1

backNforth

20

In [88]:
# find the primary language of the max=5 queries

queries = [take['message'] for take in conversation['conversation']if take['role']=='user']
queries.pop(0)

from langdetect import detect, detect_langs

langset = set()
for i in range(min(len(queries), 5)):
    lang = detect(queries[i][0])
    langset.add(lang)

list(langset)

['ja']

In [89]:
# count of characters
#   this also includes code
#   this includes markdown language... so it's not good

user_messages = queries
assistant_messages = [take['message'] for take in conversation['conversation']if take['role']=='assistant']

charcount_user = sum([len(take[0]) for take in user_messages if len(take)!=0])
charcount_assistant = sum([len(take[0]) for take in assistant_messages if len(take)!=0])

charcount_user, charcount_assistant

(2251, 18870)

In [90]:
# character count for code OR structured data

assistant_messages = [take['message'] for take in conversation['conversation']if take['role']=='assistant']

import re
texts = []
for take in assistant_messages:
    texts += take

for text in texts:
    code_blocks = re.findall(r"```(.*?)```", text, re.DOTALL)
    # Print extracted code blocks
    print([{
        'type': code_block.split('\n')[0].strip(),
        'length': len('\n'.join(code_block.split('\n')[1:]))
    } for code_block in code_blocks])

    # for code_block in code_blocks:
    #     code_type = code_block.split('\n')[0].strip()
    #     code_length = len(code_block)
    # for i, code in enumerate(code_blocks, 1):
    #     print(f"Code Block {i}:\n{code}\n")
    

[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]


In [85]:
# Tools... not finished yet... are we interested in this? 
#   we would have no access to the files anyways, and we can ask gpt to generate this...
#   all previous measurements focus on counting, which GPT is not super good at, especially at counting workds

# for i in range(10):
i=3
conversation_example = conversation_fnames[i]
with open(f'conversations/{conversation_example}', 'r') as f:
    conversation = json.load(f)

print(conversation['url'])
tool_messages = [take['message'] for take in conversation['conversation']if take['role']=='tool']
print(tool_messages)
conversation['conversation']

https://chatgpt.com/share/67307947-e754-8008-9860-6526e924dd42
[['The output of this plugin was redacted.'], ['The output of this plugin was redacted.'], ['The output of this plugin was redacted.'], ['The output of this plugin was redacted.'], ['The output of this plugin was redacted.'], ['The output of this plugin was redacted.'], ['The output of this plugin was redacted.'], ['The output of this plugin was redacted.'], ['The output of this plugin was redacted.'], ['The output of this plugin was redacted.'], ['The output of this plugin was redacted.'], ['The output of this plugin was redacted.'], ['The output of this plugin was redacted.'], ['The output of this plugin was redacted.'], ['The output of this plugin was redacted.'], ['The output of this plugin was redacted.'], ['The output of this plugin was redacted.'], ['The output of this plugin was redacted.'], ['The output of this plugin was redacted.'], ['The output of this plugin was redacted.']]


[{'role': 'system', 'message': ['']},
 {'role': 'user', 'message': ['civilization4について相談したいのですが、知識はありますか？']},
 {'role': 'assistant', 'message': []},
 {'role': 'assistant',
  'message': ['『Civilization IV』についてはある程度の知識があります！ゲームのメカニクスや戦略、プレイ方法についてもお手伝いできますので、どのような相談をしたいか教えてください。']},
 {'role': 'user',
  'message': ["本体に付属しているRye's and fall of civilizationというmodを個人的に改造しています。このmodは知っていますか？"]},
 {'role': 'assistant',
  'message': ['はい、『Rise and Fall of Civilization』（通称「RFC」）は、『Civilization IV』の人気のあるモッドの一つで、プレイヤーに特定の文明や歴史的な進行をより詳細に体験させることを目指しています。このモッドでは、各文明が歴史的に正確なタイミングで登場したり、特定の地域でのプレイが強調されたり、さまざまなシステムが追加されていることが特徴です。\n\n例えば、「ダイナスティ・システム」や「歴史的なイベント」など、プレイヤーが歴史を追いながら進行する形式が特徴的です。また、各文明には歴史的な拡張や困難が設けられており、非常に戦略的なプレイが求められます。\n\nあなたがこのモッドを改造しているということですが、具体的にどの部分をカスタマイズしているのか、またはどのような改善や変更を加えようとしているのか教えていただければ、アドバイスなどもできるかと思います！']},
 {'role': 'user',
  'message': ['modの調整点をまとめたテキストがありますが、7kbほどの大きさです。コピペするとgptのシステムに負担になってしまうでしょうか？']},
 {'role': 'assistant',
  'message': ['7KB程度のテキストは、GPTにとって十分処理可能